In [1]:
from __future__ import print_function

"""
A script to run cqed_rhf method on the formaldehyde molecule in a cc-pVDZ basis set
and plot the HOMO and LUMO+1 orbitals under strong and no-coupling to a photon
polarized along the $y-z$ axis, as shown in Figure 1 of McTague and Foley.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
from helper_cqed_rhf import *
from helper_cs_cqed_cis import *
from matplotlib import pyplot as plt

# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
psi4.core.set_output_file('output.dat', False)

numpy_memory = 2



  Memory set to   1.863 GiB by Python driver.


First get the orbitals with full $C_{2v}$ symmetry using canonical RHF

In [5]:
import fortecubeview

# options dict
options_dict = {'basis': 'cc-pVDZ',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-10,
               'd_convergence' : 1e-10}

psi4.set_options(options_dict)

molstr = """

0 1
O 0.0000000000 0.0000000000 5.91268220e-01
C 0.0000000000 0.0000000000 -5.90400099e-01
H 0.0000000000 9.32184336e-01 -1.17703144e+00
H 0.0000000000 -9.32184336e-01 -1.17703144e+00
no_reorient
symmetry c2v
"""
mol = psi4.geometry(molstr)
energy, wfn = psi4.energy("scf/cc-pVDZ", molecule=mol, return_wfn=True)
psi4.set_options({
    'CUBEPROP_TASKS': ['ORBITALS'], 
    'CUBEPROP_ORBITALS': [8, 10],
})
cube = psi4.cubeprop(wfn)
fortecubeview.plot()

CubeViewer: listing cube files from the directory .
Reading 5 cube files


/opt/conda/lib/python3.9/site-packages/fortecubeview/py3js_renderer.py:1096: FutureWarning: marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19
  values = skimage.measure.marching_cubes_lewiner(data, level * 0.995)


HTML(value='\n        <style>\n           .jupyter-widgets-output-area .output_scroll {\n                heigh…

interactive(children=(Select(description='Cube files:', options=('MO    8a (2-B2)', 'MO    8a (8-A)', 'MO    9…

Now get the orbitals when there is a photon polarized along the $y-z$ axis with coupling
vector $\lambda_{yz} = \left(0, \frac{0.2}{\sqrt{2}}, \frac{0.2}{\sqrt{2}} \right) $.
Since our CQED-RHF code does not make use of symmetry, we will set the point group to be $C_1$, even
though in reality it is $C_s$ with this polarization.  To get a plot of the resulting orbitals,
we will create a dictionary from the internal `Psi4` wavefunction object (which contains information
about the RHF orbitals), and modify this dictionary using the orbitals from our CQED-RHF calculation.
We will then use this modified wavefunction object to generate cube files and then plot the cube
files using fortecubeview.

In [6]:

# rhf/cc-pVDZ optimized geometry of formaldehyde
molstr = """

0 1
O 0.0000000000 0.0000000000 5.91268220e-01
C 0.0000000000 0.0000000000 -5.90400099e-01
H 0.0000000000 9.32184336e-01 -1.17703144e+00
H 0.0000000000 -9.32184336e-01 -1.17703144e+00
no_reorient
symmetry c1
"""

# options dict
options_dict = {'basis': 'cc-pVDZ',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-10,
               'd_convergence' : 1e-10}

psi4.set_options(options_dict)
mol = psi4.geometry(molstr)

# this wavefunction object is generated by psi4 and contains
# the canonical RHF orbitals
energy, wfn = psi4.energy("scf/cc-pVDZ", molecule=mol, return_wfn=True)

# define the magnitude of the coupling vector
l = 0.1

lam_yz = np.array([0, l/np.sqrt(2), l/np.sqrt(2)])

# run our CQED-RHF calculation and store the results in a dictionary called 'yz_dict'
yz_dict = cqed_rhf(lam_yz, molstr, options_dict)

# get the CQED-RHF orbitals from yz_dict
C_yz = yz_dict['CQED-RHF C']

# get the psi4 wavefunction as a dictionary
rhf_wfn_dict = psi4.core.Wavefunction.to_file(wfn)

# swap out orbitals in the dictionary with our orbitals from CQED-RHF
rhf_wfn_dict['matrix']['Ca'] = C_yz
rhf_wfn_dict['matrix']['Cb'] = C_yz

# Now create a new wavefunction object that has the CQED-RHF orbitals 'cqedrhf_wfn'
cqedrhf_wfn = psi4.core.Wavefunction.from_file(rhf_wfn_dict) 

# Confirm you have copied the CQED-RHF orbitals properly 
# by again getting a dic of the wavefunction and comparing
# the orbitals to the original numpy array 'C' that resulted
# from the CQED-RHF iterations
cqedrhf_wfn_dict = psi4.core.Wavefunction.to_file(cqedrhf_wfn)

assert np.allclose(cqedrhf_wfn_dict['matrix']['Ca'], C_yz, 1e-6)



# homo
HOMO = 8
# lumo+1
LUMOp1 = 10     # These are is the HOMO and LUMO+1 for formaldehyde


psi4.set_options({
    'CUBEPROP_TASKS': ['ORBITALS'], # calculate orbitals
    'CUBEPROP_ORBITALS': [HOMO, LUMOp1],
})

# create cube files using our modified wavefunction object
cube = psi4.cubeprop(cqedrhf_wfn)

# plot the cube files with fortecubeview
import fortecubeview
fortecubeview.plot(colorscheme='emory', opacity=.95)


Start SCF iterations:

Canonical RHF One-electron energy = -218.2835959999193847
CQED-RHF One-electron energy      = -218.1642419191604176
Nuclear repulsion energy          = 31.7803657982661569
Dipole energy                     = 0.0035655876892623
SCF Iteration   1: Energy = -113.8386337594120477   dE = -1.13839E+02   dRMS = 5.42191E-04
SCF Iteration   2: Energy = -113.8390449771818567   dE = -4.11218E-04   dRMS = 2.23355E-04
SCF Iteration   3: Energy = -113.8390792592478959   dE = -3.42821E-05   dRMS = 1.15465E-04
SCF Iteration   4: Energy = -113.8390851913647595   dE = -5.93212E-06   dRMS = 8.97796E-05
SCF Iteration   5: Energy = -113.8390871942177398   dE = -2.00285E-06   dRMS = 6.55615E-05
SCF Iteration   6: Energy = -113.8390881803574644   dE = -9.86140E-07   dRMS = 5.11581E-05
SCF Iteration   7: Energy = -113.8390887321005494   dE = -5.51743E-07   dRMS = 3.87993E-05
SCF Iteration   8: Energy = -113.8390890528602313   dE = -3.20760E-07   dRMS = 3.00177E-05
SCF Iteration   9: En

/opt/conda/lib/python3.9/site-packages/fortecubeview/py3js_renderer.py:1096: FutureWarning: marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19
  values = skimage.measure.marching_cubes_lewiner(data, level * 0.995)


HTML(value='\n        <style>\n           .jupyter-widgets-output-area .output_scroll {\n                heigh…

interactive(children=(Select(description='Cube files:', options=('MO    8a (2-B2)', 'MO    8a (8-A)', 'MO    9…